In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests
import re
import time

hdr = {'User-Agent': 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'}
i = 0
product_names = []
product_prices = []
product_descs = []
product_profile = []

In [2]:
# find upper limit of LCBO wine pages

url = 'https://www.lcbo.com/webapp/wcs/stores/servlet/en/lcbo/wine-14?pageView=grid&orderBy=5&fromPage=catalogEntryList&beginIndex=0'
response = requests.get(url, headers=hdr)
soup = bs(response.text, 'lxml')

##isolating web container containing all products on the page
last_page_ls = soup.find_all('a', class_="hoverover")

for x in last_page_ls:
    if int(x.text) > 2:
        last_page = int(x.text)
        break
        
print(last_page)

839


In [3]:
while i <= last_page:
    
    #setting the url to be scraped
    url = f'https://www.lcbo.com/webapp/wcs/stores/servlet/en/lcbo/wine-14?pageView=grid&orderBy=5&fromPage=catalogEntryList&beginIndex={i*12}'
    response = requests.get(url, headers=hdr)
    soup = bs(response.text, 'lxml')
    
    #isolating web container containing all products on the page
    products = soup.find('div', class_="product_listing_container")
    
    #extracting all product names
    names = products.find_all('div', class_="product_name")
    product_names.extend(name.text.replace('\n','') for name in names)
    
    #extracting all product prices
    prices = products.find_all('span', class_="price")
    product_prices.extend(float(price.text.replace(',','').replace('\n', '').replace('\t','').replace('$','')) for price in prices)

    #extracting description urls
    product_descs.extend(str(name.find('a')['href']) for name in names)
    
    #creating delay
    #time.sleep(2.5)
    
    #increase page count
    i+=1

print("it's done!")

it's done!


In [4]:
df1 = pd.DataFrame(data = {'Wine': product_names,
                           'Price': product_prices,
                           'URL': product_descs})

df1.head()

,Wine,Price,URL
0,Sogrape Mateus Rosé,8.95,https://www.lcbo.com/webapp/wcs/stores/servlet...
1,Mommessin Cuvee St Pierre Rouge,8.45,https://www.lcbo.com/webapp/wcs/stores/servlet...
2,Chiarli Castelvetro Lambrusco,10.30,https://www.lcbo.com/webapp/wcs/stores/servlet...
3,Blue Nun Deutscher Tafelwein,9.45,https://www.lcbo.com/webapp/wcs/stores/servlet...
4,Folonari Valpolicella Classico DOC,14.95,https://www.lcbo.com/webapp/wcs/stores/servlet...


In [5]:
start = time.time()
for link in df1['URL']:
    url = link
    response = requests.get(url, headers=hdr)
    soup = bs(response.text, 'lxml')
    
    #extracting description with empty description contingency
    if soup.find('p', class_="hidden-xs"):
        text = soup.find('p', class_= "hidden-xs").text
    else:
        text = 'N/A'
        
    #appending descriptions to list
    product_profile.append(text)
end = time.time()

print(end-start)

7195.260917186737


In [7]:
print(product_profile[10067])

Garnet in colour, this red boasts a medium nose exuding woody, spicy and fruity aromas. Discover this medium-bodied wine expressing refreshing acidity and offering a broad texture and fleshy tannins. It also unveils a medium mouthfeel that leads into medium finish.


In [8]:
print(df1['URL'][10067])

https://www.lcbo.com/webapp/wcs/stores/servlet/en/lcbo/wine-14/ch%C3%A2teau-st-thomas-les-%C3%A9mirs-2013-840421


In [9]:
df1


,Wine,Price,URL
0,Sogrape Mateus Rosé,8.95,https://www.lcbo.com/webapp/wcs/stores/servlet...
1,Mommessin Cuvee St Pierre Rouge,8.45,https://www.lcbo.com/webapp/wcs/stores/servlet...
2,Chiarli Castelvetro Lambrusco,10.30,https://www.lcbo.com/webapp/wcs/stores/servlet...
3,Blue Nun Deutscher Tafelwein,9.45,https://www.lcbo.com/webapp/wcs/stores/servlet...
4,Folonari Valpolicella Classico DOC,14.95,https://www.lcbo.com/webapp/wcs/stores/servlet...
5,Mouton Cadet Bordeaux Red AOC,16.45,https://www.lcbo.com/webapp/wcs/stores/servlet...
6,Andrès Baby Duck,12.60,https://www.lcbo.com/webapp/wcs/stores/servlet...
7,Fontana Di Papa White,14.70,https://www.lcbo.com/webapp/wcs/stores/servlet...
8,Flat Rock Pinot Noir,20.95,https://www.lcbo.com/webapp/wcs/stores/servlet...
9,Flat Rock Twisted White,17.95,https://www.lcbo.com/webapp/wcs/stores/servlet...


In [11]:
df1.pop('URL')

0        https://www.lcbo.com/webapp/wcs/stores/servlet...
1        https://www.lcbo.com/webapp/wcs/stores/servlet...
2        https://www.lcbo.com/webapp/wcs/stores/servlet...
3        https://www.lcbo.com/webapp/wcs/stores/servlet...
4        https://www.lcbo.com/webapp/wcs/stores/servlet...
5        https://www.lcbo.com/webapp/wcs/stores/servlet...
6        https://www.lcbo.com/webapp/wcs/stores/servlet...
7        https://www.lcbo.com/webapp/wcs/stores/servlet...
8        https://www.lcbo.com/webapp/wcs/stores/servlet...
9        https://www.lcbo.com/webapp/wcs/stores/servlet...
10       https://www.lcbo.com/webapp/wcs/stores/servlet...
11       https://www.lcbo.com/webapp/wcs/stores/servlet...
12       https://www.lcbo.com/webapp/wcs/stores/servlet...
13       https://www.lcbo.com/webapp/wcs/stores/servlet...
14       https://www.lcbo.com/webapp/wcs/stores/servlet...
15       https://www.lcbo.com/webapp/wcs/stores/servlet...
16       https://www.lcbo.com/webapp/wcs/stores/servlet.

In [14]:
desc_series = pd.Series(product_profile, name = 'Description')
df2 = df1.join(desc_series)

In [15]:
df2.head()

,Wine,Price,Description
0,Sogrape Mateus Rosé,8.95,"Mateus Rosé, in the iconic flask-shaped bottle..."
1,Mommessin Cuvee St Pierre Rouge,8.45,This red blend is pale ruby in colour and offe...
2,Chiarli Castelvetro Lambrusco,10.30,Lambrusco is a specialty of Italy's Emilia-Rom...
3,Blue Nun Deutscher Tafelwein,9.45,This top-selling wine phenomenon is back in a ...
4,Folonari Valpolicella Classico DOC,14.95,Valpolicella is the classic red wine of Italy'...


In [16]:
#Setting the bag of words for each wine characteristic

body = {
    "light body": ["thin", "cliff-edge", "hollow", "short", "delicate", "elegant"],
    "medium body": ["mellow", "medium-bodied"],
    "full body": ["fat", "dense", "opulent", "rich", "thick"]
}

tann = {
    "low tannins": ["silky", "rounded", "powdery", "supple", "smooth"],
    "medium tannins": ["velvet", "plush", "fine grained", "polished", "sinewy", "chalky", "grainy"],
    "high tannins": ["chewy", "chunky", "bulky", "muscular", "grippy", "angular"]
}

fruit = {
    "fruity": ["fruity","melon","apple","jammy", "ripe", "juicy", "flamboyant", "fleshy", "extracted", "plumy", "red fruit", "dark fruit", "grapey", "cassis"],
    "berry": ["strawberry", "raspberry", "cherry", "blueberry", "blackberry"],
    "citrus": ["lime", "lemon", "grapefruit", "orange", "citrus zest", "citrus"],
    "stone fruit": ["apricot", "nectarine", "peach"],
    "tropical fruit": ["tropical fruit","banana", "Pineapple", "leechie", "coconut", "papaya"]
}

acid = {
    "high acidity": ["bright", "lean", "racy", "tart", "angular", "edgy", "nerve", "zippy", "zesty", "lively", "fresh", "crisp", "brisk"],
    "low acidity": ["soft", "flabby", "flat", "delicate", "fallen over"]
}

In [17]:
#Creating the classifier function
def rev_class(review):
    
    #Default setting for each characteristic
    b_term = "N/A"
    t_term = "N/A"
    f_term = "N/A"
    a_term = "N/A"
    
    #Loop through bag of words to classify body
    for char0, param0 in body.items():
        for x in param0:
            if x in review:
                b_term = char0
                break
        
    #Loop through bag of words to classify tannin content        
    for char1, param1 in tann.items():
        for i in param1:
            if i in review:
                t_term = char1
                break

    
    #Loop through bag of words to classify fruitiness
    for char2, param2 in fruit.items():
        for j in param2:
            if j in review:
                f_term = char2
                break

    #Loop through bag of words to classify acid content
    for char3, param3 in acid.items():
        for k in param3:
            if k in review:
                a_term = char3
                break
    
    return b_term, t_term, f_term, a_term

In [18]:
body_ls = []
tannin_ls = []
fruit_ls = []
acid_ls = []

for review in df2['Description']:
    b_term, t_term, f_term, a_term = rev_class(review)
    
    body_ls.append(b_term)
    tannin_ls.append(t_term)
    fruit_ls.append(f_term)
    acid_ls.append(a_term)
    
print(len(body_ls), len(tannin_ls), len(fruit_ls), len(acid_ls))

10068 10068 10068 10068


In [19]:
body_series = pd.Series(body_ls, name = 'Body')
tannin_series = pd.Series(tannin_ls, name = 'Tannins')
fruit_series = pd.Series(fruit_ls, name = "Fruit")
acid_series = pd.Series(acid_ls, name = "Acid")

df3 = df2.join(body_series)
df4 = df3.join(tannin_series)
df5 = df4.join(fruit_series)
df_final = df5.join(acid_series)

In [20]:
df_final

,Wine,Price,Description,Body,Tannins,Fruit,Acid
0,Sogrape Mateus Rosé,8.95,"Mateus Rosé, in the iconic flask-shaped bottle...",N/A,N/A,berry,low acidity
1,Mommessin Cuvee St Pierre Rouge,8.45,This red blend is pale ruby in colour and offe...,N/A,N/A,berry,N/A
2,Chiarli Castelvetro Lambrusco,10.30,Lambrusco is a specialty of Italy's Emilia-Rom...,full body,N/A,berry,N/A
3,Blue Nun Deutscher Tafelwein,9.45,This top-selling wine phenomenon is back in a ...,light body,N/A,stone fruit,low acidity
4,Folonari Valpolicella Classico DOC,14.95,Valpolicella is the classic red wine of Italy'...,N/A,low tannins,berry,low acidity
5,Mouton Cadet Bordeaux Red AOC,16.45,"From France's leading wine family, a wine that...",N/A,N/A,N/A,N/A
6,Andrès Baby Duck,12.60,"Launched in the 1970s, Baby Duck once dominate...",N/A,N/A,fruity,high acidity
7,Fontana Di Papa White,14.70,The grapes for this light and refreshing wine ...,light body,N/A,citrus,low acidity
8,Flat Rock Pinot Noir,20.95,Innovative Flat Rock Cellars specializes in cr...,light body,N/A,berry,N/A
9,Flat Rock Twisted White,17.95,N/A,N/A,N/A,N/A,N/A


In [22]:
df_final.pop('Description')

0        Mateus Rosé, in the iconic flask-shaped bottle...
1        This red blend is pale ruby in colour and offe...
2        Lambrusco is a specialty of Italy's Emilia-Rom...
3        This top-selling wine phenomenon is back in a ...
4        Valpolicella is the classic red wine of Italy'...
5        From France's leading wine family, a wine that...
6        Launched in the 1970s, Baby Duck once dominate...
7        The grapes for this light and refreshing wine ...
8        Innovative Flat Rock Cellars specializes in cr...
9                                                      N/A
10       Ruffino's original wine, and one of the world'...
11       Meticulous vineyard practices ensure optimum f...
12       This vivacious sparkler has been a favourite i...
13       Susana Balbo has been called the 'Evita of Win...
14       From France's leading wine family, a wine that...
15       The mischievous name of this wine is an old ex...
16       The grapes for this food-friendly wine are gro.

In [23]:
df_final

,Wine,Price,Body,Tannins,Fruit,Acid
0,Sogrape Mateus Rosé,8.95,N/A,N/A,berry,low acidity
1,Mommessin Cuvee St Pierre Rouge,8.45,N/A,N/A,berry,N/A
2,Chiarli Castelvetro Lambrusco,10.30,full body,N/A,berry,N/A
3,Blue Nun Deutscher Tafelwein,9.45,light body,N/A,stone fruit,low acidity
4,Folonari Valpolicella Classico DOC,14.95,N/A,low tannins,berry,low acidity
5,Mouton Cadet Bordeaux Red AOC,16.45,N/A,N/A,N/A,N/A
6,Andrès Baby Duck,12.60,N/A,N/A,fruity,high acidity
7,Fontana Di Papa White,14.70,light body,N/A,citrus,low acidity
8,Flat Rock Pinot Noir,20.95,light body,N/A,berry,N/A
9,Flat Rock Twisted White,17.95,N/A,N/A,N/A,N/A


In [24]:
df_final.to_csv (r'C:\Users\Llobo\Documents\Data_analytics\Wine AI\final_lcbo.csv', index = None)